In [4]:
import pandas as pd
import random

# CSV 파일 로드
file_path = "../../data/train.csv"  # 실제 파일 경로 지정
df = pd.read_csv(file_path)

# 📌 질문지 생성 함수
def generate_questions(row):
    questions = []

    # 1️⃣ 사고 발생 상황 기반 질문
    if pd.notna(row["발생일시"]) and pd.notna(row["날씨"]) and pd.notna(row["기온"]) and pd.notna(row["습도"]):
        questions.append(
            f"{row['발생일시']}에 {row['날씨']} 상태에서 {row['기온']}도, 습도 {row['습도']}% 환경에서 발생한 사고의 원인은?"
        )

    # 2️⃣ 공사 관련 질문
    if pd.notna(row["공사종류"]) and pd.notna(row["연면적"]) and pd.notna(row["층 정보"]):
        questions.append(
            f"{row['공사종류']}에서 연면적 {row['연면적']}㎡, {row['층 정보']}층에서 발생한 사고의 원인은?"
        )

    # 3️⃣ 사고 유형 기반 질문
    if pd.notna(row["인적사고"]) or pd.notna(row["물적사고"]):
        accident_desc = []
        if pd.notna(row["인적사고"]):
            accident_desc.append(f"인적사고({row['인적사고']})")
        if pd.notna(row["물적사고"]):
            accident_desc.append(f"물적사고({row['물적사고']})")
        questions.append(
            f"{' 및 '.join(accident_desc)}가 발생한 사고에서의 대응 방안은?"
        )

    # 4️⃣ 작업 프로세스 관련 질문
    if pd.notna(row["작업프로세스"]) and pd.notna(row["사고객체"]) and pd.notna(row["장소"]):
        questions.append(
            f"{row['장소']}에서 {row['작업프로세스']} 중 {row['사고객체']}로 인해 발생한 사고의 원인은?"
        )

    # 5️⃣ 사고원인 기반 질문
    if pd.notna(row["사고원인"]):
        questions.append(
            f"{row['사고원인']}으로 인해 발생한 사고의 예방 대책은?"
        )

    return questions

# 📌 질문지 생성
question_data = []
for _, row in df.iterrows():
    questions = generate_questions(row)
    for question in questions:
        question_data.append({"ID": row["ID"], "question": question})

# DataFrame 생성
question_df = pd.DataFrame(question_data)

# 📌 랜덤 샘플 10개 출력
print(question_df.sample(10))

# CSV로 저장 (추후 RAG 입력 데이터로 활용 가능)
# output_path = "data/generated_questions.csv"
# question_df.to_csv(output_path, index=False, encoding="utf-8-sig")

# print(f"✅ 질문지 생성 완료 → {output_path}")


                 ID                                           question
33468   TRAIN_06702      운수시설 / 외부에서 설치작업 중 건설자재 > 자재로 인해 발생한 사고의 원인은?
19482   TRAIN_03896   업무시설 / 외부에서 형틀 및 목공 중 건설자재 > 철근로 인해 발생한 사고의 원인은?
34343   TRAIN_06878  2022-11-14 오전 11:50에 맑음 상태에서 10℃도, 습도 30%% 환경에...
48275   TRAIN_09669   공동주택 / 외부에서 운반작업 중 건설기계 > 타워크레인로 인해 발생한 사고의 원인은?
108585  TRAIN_21830                  사다리 작업 중 부주의으로 인해 발생한 사고의 예방 대책은?
67537   TRAIN_13523  공동주택 / 내부에서 형틀 및 목공 중 건설공구 > 공구류로 인해 발생한 사고의 원인은?
12543   TRAIN_02508      하수도 / 내부에서 굴착작업 중 기타 > 지하매설물로 인해 발생한 사고의 원인은?
50490   TRAIN_10112     업무시설 / 내부에서 설치작업 중 가시설 > 작업발판로 인해 발생한 사고의 원인은?
60819   TRAIN_12178      공동주택 / 내부에서 설치작업 중 건설자재 > 자재로 인해 발생한 사고의 원인은?
35865   TRAIN_07182  지하주차장으로 자재를 하역중 타워크레인으로 자재를 받기위해 이동중 반출중인 톤마대에...


In [2]:
text = '테스트 - 1 -\n\n langchain이란? - 2 -'

def page_split(content):
    '''page별로 쪼개버림'''
    page_dict = {}
    i = 0
    while True:
        i += 1
        contents = content.split(f"- {i} -")
        if len(contents) != 2:
            break
        page_dict[str(i)] = contents[0]
        content = contents[1]
    return page_dict

page_split(text)

{'1': '테스트 ', '2': '\n\n langchain이란? '}

In [5]:
import re
tmp = re.split(r"-\s(\d+)\s-", text)
print(tmp)
result = {tmp[i]: tmp[i-1] for i in range(1, len(tmp)-1, 2)}
print(result)

['테스트 ', '1', '\n\n langchain이란? ', '2', '']
{'1': '테스트 ', '2': '\n\n langchain이란? '}
